In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta

import cufflinks as cf
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import seaborn as sns
import plotly.express as px
%matplotlib inline

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

In [2]:
df_main = pd.read_csv('snow3300.csv')
pd.set_option('max_columns', None)

# TO INCLUDE ONLY RESULTS AFTER OCTOBER, USE THIS
df_main = df_main[ df_main['opened_at'] > '2020-10-01' ]

# Change datatypes of all time-based columns
time_columns = ['sys_updated_on', 'sys_created_on', 'closed_at', 'opened_at', 'business_duration', 'reopened_time',
                'resolved_at', 'work_start', 'calendar_duration']

for item in time_columns:
    df_main[item] = pd.to_datetime(df_main[item])

# Create additional time columns
df_main['opened_at_hour'] = df_main['opened_at'].dt.hour
df_main['opened_at_month'] = df_main['opened_at'].dt.month
df_main['opened_at_day'] = df_main['opened_at'].dt.day
df_main['opened_at_day_of_year'] = df_main['opened_at'].dt.dayofyear
df_main['opened_at_clone'] = df_main['opened_at']
df_main['resolution_time_length'] = df_main.resolved_at - df_main.opened_at
df_main['resolution_time_total_hours'] = df_main['resolution_time_length'].dt.total_seconds()/3600
# print(type(df_main['opened_at_clone'].iloc[0]))
df_main[['opened_at', 'task_effective_number', 'opened_at_hour', 'opened_at_month', 'opened_at_day', 'opened_at_day_of_year',
         'opened_at_clone', 'resolution_time_length', 'resolution_time_total_hours']].head(5).sort_values('opened_at_day').head(2)

,opened_at,task_effective_number,opened_at_hour,opened_at_month,opened_at_day,opened_at_day_of_year,opened_at_clone,resolution_time_length,resolution_time_total_hours
5,2020-11-02 16:02:50,INC0053368,16,11,2,307,2020-11-02 16:02:50,1 days 23:49:28,47.824444
4,2020-12-03 21:29:25,INC0056332,21,12,3,338,2020-12-03 21:29:25,0 days 01:06:38,1.110556


In [3]:
# "Fix" time columns by correcting the time    
df_main['opened_at'] = df_main['opened_at'] - timedelta(hours=5)
df_main['resolved_at'] = df_main['resolved_at'] - timedelta(hours=5)
df_main['sys_updated_on'] = df_main['sys_updated_on'] - timedelta(hours=5)
df_main['sys_created_on'] = df_main['sys_created_on'] - timedelta(hours=5)
df_main['closed_at'] = df_main['closed_at'] - timedelta(hours=5)
df_main['business_duration'] = df_main['business_duration'] - timedelta(hours=5)
df_main['reopened_time'] = df_main['reopened_time'] - timedelta(hours=5)
df_main['work_start'] = df_main['work_start'] - timedelta(hours=5)
df_main['calendar_duration'] = df_main['calendar_duration'] - timedelta(hours=5)

In [4]:
df_support = df_main[ (df_main['resolved_by_username_value'] == 'Shawn McCoy') |
                     (df_main['resolved_by_username_value'] == 'Taylor Ramsey')|
                    (df_main['resolved_by_username_value'] == 'Valentin Correa')]

# df_main['opened_at'].groupby(df_main['resolved_by_username_value']).count().sort_values(ascending=False)

In [5]:
df_support['resolved_by_username_value'].unique()

array(['Shawn McCoy', 'Taylor Ramsey', 'Valentin Correa'], dtype=object)

In [6]:
df_support['resolved_by_username_value']

2           Shawn McCoy
4           Shawn McCoy
5           Shawn McCoy
8           Shawn McCoy
13          Shawn McCoy
             ...       
3296    Valentin Correa
3297    Valentin Correa
3298    Valentin Correa
3299    Valentin Correa
3300    Valentin Correa
Name: resolved_by_username_value, Length: 2063, dtype: object

In [7]:
df_support['opened_at'].dt.day.groupby(df_support['resolved_by_username_value']).count()

resolved_by_username_value
Shawn McCoy         672
Taylor Ramsey      1109
Valentin Correa     282
Name: opened_at, dtype: int64

In [8]:
df_support['resolved_by_username_value'].groupby(df_support['opened_at'].dt.day)

In [9]:
# for a monthly average
df_main['task_effective_number'].groupby(df_support['opened_at'].dt.day).count()

opened_at
1.0      69
2.0     117
3.0      82
4.0      88
5.0      88
6.0      44
7.0      42
8.0      95
9.0      94
10.0     53
11.0     71
12.0     66
13.0     53
14.0     70
15.0     68
16.0     81
17.0     65
18.0     45
19.0     86
20.0     75
21.0     66
22.0     93
23.0     59
24.0     49
25.0     72
26.0     66
27.0     47
28.0     51
29.0     50
30.0     51
31.0      7
Name: task_effective_number, dtype: int64

In [10]:
df_main['opened_at'].sort_values()

2088   2020-10-01 07:59:03
711    2020-10-01 10:38:05
2945   2020-10-01 10:58:48
2606   2020-10-01 11:06:46
2789   2020-10-01 11:43:17
               ...        
2605   2021-03-09 12:55:23
2446   2021-03-09 13:54:21
2371   2021-03-09 14:05:30
723    2021-03-09 14:33:46
2476   2021-03-09 15:27:40
Name: opened_at, Length: 2263, dtype: datetime64[ns]

In [11]:
# df_support = df_support[ df_support['opened_at'] < '2021-01-17' ]
df_support = df_support[ df_support['opened_at'] > '2020-10-01' ]
df_support['opened_at'].dt.day.groupby(df_support['resolved_by_username_value']).count()

resolved_by_username_value
Shawn McCoy         672
Taylor Ramsey      1109
Valentin Correa     282
Name: opened_at, dtype: int64

In [12]:
df_taylor = df_support[ df_support['resolved_by_username_value'] == 'Taylor Ramsey' ]
df_valentin = df_support[ df_support['resolved_by_username_value'] == 'Valentin Correa' ]
df_shawn = df_support[ df_support['resolved_by_username_value'] == 'Shawn McCoy' ]

# inc_daily_taylor = df_taylor['task_effective_number'].groupby(df_support['opened_at'].dt.dayofyear).count()
# inc_daily_valentin = df_valentin['task_effective_number'].groupby(df_support['opened_at'].dt.dayofyear).count()
# inc_daily_shawn = df_shawn['task_effective_number'].groupby(df_support['opened_at'].dt.dayofyear).count()

inc_daily_taylor = df_taylor['task_effective_number'].groupby(df_support['opened_at'].dt.date).count()
inc_daily_valentin = df_valentin['task_effective_number'].groupby(df_support['opened_at'].dt.date).count()
inc_daily_shawn = df_shawn['task_effective_number'].groupby(df_support['opened_at'].dt.date).count()

inc_daily_data = {"taylor": inc_daily_taylor,
                "valentin": inc_daily_valentin,
                "shawn": inc_daily_shawn}

df = pd.concat(inc_daily_data,axis = 1)
df = df.fillna(0)
df = df.sort_index()
df.tail(2)

,taylor,valentin,shawn
opened_at,,,
2021-03-08,11.0,8.0,9.0
2021-03-09,6.0,2.0,5.0


In [13]:
fig = px.line(df, x=df.index, y=[df.taylor, df.shawn, df.valentin], title='Count of INC resolved per day by agent')
fig.show()

In [14]:
df_avg_baseline = df_main[ (df_main['resolved_by_username_value'] == 'Shawn McCoy') |
                     (df_main['resolved_by_username_value'] == 'Taylor Ramsey')|
                    (df_main['resolved_by_username_value'] == 'Valentin Correa')]


averag_inc_resolved = (df_avg_baseline['opened_at'].dt.day.groupby(df_support['resolved_by_username_value']).mean()[0] +
df_avg_baseline['opened_at'].dt.day.groupby(df_support['resolved_by_username_value']).mean()[1] +
df_avg_baseline['opened_at'].dt.day.groupby(df_support['resolved_by_username_value']).mean()[2]) / 3

print(averag_inc_resolved)

14.39807396020524


In [15]:
df_avg_baseline['opened_at'].dt.day.groupby(df_support['resolved_by_username_value']).mean()

resolved_by_username_value
Shawn McCoy        14.614583
Taylor Ramsey      14.675383
Valentin Correa    13.904255
Name: opened_at, dtype: float64

In [16]:
df_support = df_main[ (df_main['resolved_by_username_value'] == 'Shawn McCoy') |
                     (df_main['resolved_by_username_value'] == 'Taylor Ramsey')|
                    (df_main['resolved_by_username_value'] == 'Valentin Correa')]

# start_time & end_time
df_support = df_support[ (df_support['resolved_at'] >= '2021-02-01')  ]
df_support['resolved_at'].dt.day.groupby(df_support['resolved_by_username_value']).count()

df_taylor = df_support[ df_support['resolved_by_username_value'] == 'Taylor Ramsey' ]
df_valentin = df_support[ df_support['resolved_by_username_value'] == 'Valentin Correa' ]
df_shawn = df_support[ df_support['resolved_by_username_value'] == 'Shawn McCoy' ]

inc_daily_taylor = df_taylor['task_effective_number'].groupby(df_support['resolved_at'].dt.date).count()
inc_daily_valentin = df_valentin['task_effective_number'].groupby(df_support['resolved_at'].dt.date).count()
inc_daily_shawn = df_shawn['task_effective_number'].groupby(df_support['resolved_at'].dt.date).count()

inc_daily_data = {"taylor": inc_daily_taylor,
                "valentin": inc_daily_valentin,
                "shawn": inc_daily_shawn}

df = pd.concat(inc_daily_data,axis = 1)
df = df.fillna(0)
df = df.sort_index()


# THIS IS THE CORRECT WAY TO BUILD THE CHART, USING 'OPENED AT' WILL ADD WEEKEND DAYS (IF INC OPENED THAT DAY)

averag_inc_resolved = (df_support['resolved_at'].dt.day.groupby(df_support['resolved_by_username_value']).mean()[0] +
df_support['resolved_at'].dt.day.groupby(df_support['resolved_by_username_value']).mean()[1] +
df_support['resolved_at'].dt.day.groupby(df_support['resolved_by_username_value']).mean()[2]) / 3

fig = px.line(df, x=df.index, y=[df.taylor, df.shawn, df.valentin], title='Count of INC resolved per day by agent')
fig.add_hline(y=averag_inc_resolved)
fig.show()

In [17]:
averag_inc_resolved =(df_support['resolved_at'].dt.day.groupby(df_support['resolved_by_username_value']).count()[0]+
df_support['resolved_at'].dt.day.groupby(df_support['resolved_by_username_value']).count()[1]+
df_support['resolved_at'].dt.day.groupby(df_support['resolved_by_username_value']).count()[2]) / len(df_support['opened_at_clone'].dt.date.unique())

print(averag_inc_resolved)
print(len(df_support['opened_at_clone'].dt.date.unique()))

16.444444444444443
36


In [18]:
df_avg_baseline['resolved_at'].dt.day.groupby(df_support['resolved_by_username_value']).count()

resolved_by_username_value
Shawn McCoy        137
Taylor Ramsey      274
Valentin Correa    181
Name: resolved_at, dtype: int64

In [19]:
df_avg_baseline = df_support[ (df_support['resolved_by_username_value'] == 'Shawn McCoy') |
                     (df_support['resolved_by_username_value'] == 'Taylor Ramsey')|
                    (df_support['resolved_by_username_value'] == 'Valentin Correa')]
df_avg_baseline['opened_at'].dt.day.groupby(df_support['resolved_by_username_value']).mean()

resolved_by_username_value
Shawn McCoy        11.605839
Taylor Ramsey      11.357664
Valentin Correa    11.447514
Name: opened_at, dtype: float64

In [20]:
df_avg_baseline = df_support[ (df_support['resolved_by_username_value'] == 'Shawn McCoy') |
                     (df_support['resolved_by_username_value'] == 'Taylor Ramsey')|
                    (df_support['resolved_by_username_value'] == 'Valentin Correa')]


# THIS THINKING MAY BE ALL WRONG! IT"S NOT THE RESOLVED BY I SHOULD BE REVIEWING HERE
# ALSO, THE CHART BELOW IS MISSING ONLY SATURDAYS

averag_inc_resolved = (df_avg_baseline['opened_at'].dt.day.groupby(df_support['resolved_by_username_value']).mean()[0] +
df_avg_baseline['opened_at'].dt.day.groupby(df_support['resolved_by_username_value']).mean()[1] +
df_avg_baseline['opened_at'].dt.day.groupby(df_support['resolved_by_username_value']).mean()[2]) / 3

averag_inc_resolved

11.470339153929912

In [21]:
averag_inc_resolved = (df_avg_baseline['opened_at'].dt.day.groupby(df_support['resolved_by_username_value']).mean()[0] +
df_avg_baseline['opened_at'].dt.day.groupby(df_support['resolved_by_username_value']).mean()[1] +
df_avg_baseline['opened_at'].dt.day.groupby(df_support['resolved_by_username_value']).mean()[2]) / 3
averag_inc_resolved

11.470339153929912

In [22]:
df_avg_baseline['opened_at'].dt.day.groupby(df_support['resolved_by_username_value']).mean()[0]

11.605839416058394

In [23]:
df_avg_baseline['opened_at'].dt.day.groupby(df_support['resolved_by_username_value']).mean()

resolved_by_username_value
Shawn McCoy        11.605839
Taylor Ramsey      11.357664
Valentin Correa    11.447514
Name: opened_at, dtype: float64

In [24]:
averag_inc_resolved

11.470339153929912

### Count of INC resolved per day per supprot member (with an "average" bar added)

In [25]:
df_support = df_main[ (df_main['resolved_by_username_value'] == 'Shawn McCoy') |
                     (df_main['resolved_by_username_value'] == 'Taylor Ramsey')|
                    (df_main['resolved_by_username_value'] == 'Valentin Correa')]

# start_time & end_time
df_support = df_support[ (df_support['resolved_at'] >= '2021-02-28')  ]
df_support['resolved_at'].dt.day.groupby(df_support['resolved_by_username_value']).count()

df_taylor = df_support[ df_support['resolved_by_username_value'] == 'Taylor Ramsey' ]
df_valentin = df_support[ df_support['resolved_by_username_value'] == 'Valentin Correa' ]
df_shawn = df_support[ df_support['resolved_by_username_value'] == 'Shawn McCoy' ]

inc_daily_taylor = df_taylor['task_effective_number'].groupby(df_support['resolved_at'].dt.date).count()
inc_daily_valentin = df_valentin['task_effective_number'].groupby(df_support['resolved_at'].dt.date).count()
inc_daily_shawn = df_shawn['task_effective_number'].groupby(df_support['resolved_at'].dt.date).count()

inc_daily_data = {"taylor": inc_daily_taylor,
                "valentin": inc_daily_valentin,
                "shawn": inc_daily_shawn}

df = pd.concat(inc_daily_data,axis = 1)
df = df.fillna(0)
df = df.sort_index()

df_avg_baseline = df_main[ (df_main['resolved_by_username_value'] == 'Shawn McCoy') |
                     (df_main['resolved_by_username_value'] == 'Taylor Ramsey')|
                    (df_main['resolved_by_username_value'] == 'Valentin Correa')]


averag_inc_resolved = (df_avg_baseline['resolved_at'].dt.day.groupby(df_support['resolved_by_username_value']).mean()[0] +
df_avg_baseline['resolved_at'].dt.day.groupby(df_support['resolved_by_username_value']).mean()[1] +
df_avg_baseline['resolved_at'].dt.day.groupby(df_support['resolved_by_username_value']).mean()[2]) / 3

fig = px.line(df, x=df.index, y=[df.taylor, df.shawn, df.valentin], title='Count of INC per day for each agent')
fig.add_hline(y=averag_inc_resolved)
fig.show()

In [26]:
averag_inc_resolved

4.766749407100634

# find INC, resolved at certain day, by certain person, and then count them manually to make sure info is correct

In [27]:
df_avg_baseline['opened_at'].dt.day.groupby(df_support['resolved_by_username_value']).mean()

resolved_by_username_value
Shawn McCoy        8.687500
Taylor Ramsey      5.876712
Valentin Correa    5.372093
Name: opened_at, dtype: float64